In [88]:
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup
import requests
import json 
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time 
from datetime import datetime,timedelta
from scipy.stats import norm
from tabulate import tabulate

In [89]:
# IF YOU ARE SELECTING NIFTY ,BANKNIFTY,MIDCAP  SELECT SEW[0] FOR INDICES ELSE FOR STOCK SELECT SWG[1] 
symbol="NIFTY"
select_swg=["indices","equities"]
seg=select_swg[0]
EX="29-Feb-2024" #enter the expiry date 

In [90]:

chrome_options = Options()


headers = {
    ":authority":"www.nseindia.com",
    ":method":"GET",
    ":scheme":"https",
    "Accept":"*/*",
    "Accept-Encoding":"gzip, deflate, br" ,
    "Accept-Language":"en-GB,en-US;q=0.9,en;q=0.8,hi;q=0.7",
    "Referer":"https://www.nseindia.com/option-chain",
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"
    
}

for key, value in headers.items():
    chrome_options.add_argument(f"--header={key}:{value}")


driver = webdriver.Chrome(options=chrome_options)


driver.get("https://www.nseindia.com/option-chain")

driver.get(f"https://www.nseindia.com/api/option-chain-{seg}?symbol={symbol}")
#driver.get(f"https://www.nseindia.com/api/option-chain-equities?symbol={symbol}")
json_content = driver.page_source
soup = BeautifulSoup(json_content, 'html.parser')

In [91]:
pre_tag = soup.find('pre')
if pre_tag:
    json_content = pre_tag.string.strip()
    print("works")
    driver.close()
    data_dict = json.loads(json_content)
else:
    print("No <pre> tag found in the HTML content.")

works


In [92]:
a=data_dict['records']['data']

In [93]:
ocdata=[]

In [94]:
for i in a:
    for j,k in i.items():
        if j=="CE" or j=="PE":
            info=k
            info["instrumentTyoe"]=j
            ocdata.append(info)


In [95]:
print(len(ocdata))

1278


In [96]:
df=pd.DataFrame(ocdata)
df.drop(columns=['identifier'],inplace=True)

In [97]:
df=df.sort_values(by='expiryDate')

In [98]:
df.reset_index(drop=True,inplace=True)

In [99]:
df['expiryDate']=pd.to_datetime(df['expiryDate'])
df['expiryDate'] = df['expiryDate'].apply(lambda x: pd.to_datetime(x).to_pydatetime())

In [100]:
df

,strikePrice,expiryDate,underlying,openInterest,changeinOpenInterest,pchangeinOpenInterest,totalTradedVolume,impliedVolatility,lastPrice,change,pChange,totalBuyQuantity,totalSellQuantity,bidQty,bidprice,askQty,askPrice,underlyingValue,instrumentTyoe
0,22400,2024-04-04,NIFTY,0.0,0,0.0,0,0.00,0.00,0.00,0.000000,900,950,900,375.15,50,405.30,22237.4,PE
1,22700,2024-04-04,NIFTY,0.0,0,0.0,0,0.00,0.00,0.00,0.000000,2700,2700,2700,265.35,900,288.80,22237.4,CE
2,23650,2024-04-04,NIFTY,0.0,0,0.0,1,16.29,118.75,36.65,44.640682,7300,950,50,48.00,50,79.95,22237.4,CE
3,22950,2024-04-04,NIFTY,0.0,0,0.0,1,13.15,205.70,-24.50,-10.642919,50,50,50,178.95,50,225.50,22237.4,CE
4,20650,2024-04-04,NIFTY,0.0,0,0.0,0,0.00,0.00,0.00,0.000000,4500,0,750,5.10,0,0.00,22237.4,PE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1273,25000,2026-12-31,NIFTY,0.0,0,0.0,0,0.00,0.00,0.00,0.000000,600,0,250,675.05,0,0.00,22237.4,PE
1274,19000,2026-12-31,NIFTY,4.0,0,0.0,0,0.00,0.00,0.00,0.000000,50,100,50,4532.00,50,7300.00,22237.4,CE
1275,19000,2026-12-31,NIFTY,65.0,0,0.0,0,0.00,0.00,0.00,0.000000,950,50,600,107.40,50,450.00,22237.4,PE
1276,26000,2026-12-31,NIFTY,0.0,0,0.0,0,0.00,0.00,0.00,0.000000,100,0,100,61.00,0,0.00,22237.4,CE


In [101]:
df['expiryDate']==EX

0       False
1       False
2       False
3       False
4       False
        ...  
1273    False
1274    False
1275    False
1276    False
1277    False
Name: expiryDate, Length: 1278, dtype: bool

In [102]:
df[df['instrumentTyoe']=='CE'].head(2)

,strikePrice,expiryDate,underlying,openInterest,changeinOpenInterest,pchangeinOpenInterest,totalTradedVolume,impliedVolatility,lastPrice,change,pChange,totalBuyQuantity,totalSellQuantity,bidQty,bidprice,askQty,askPrice,underlyingValue,instrumentTyoe
1,22700,2024-04-04,NIFTY,0.0,0,0.0,0,0.00,0.00,0.00,0.000000,2700,2700,2700,265.35,900,288.80,22237.4,CE
2,23650,2024-04-04,NIFTY,0.0,0,0.0,1,16.29,118.75,36.65,44.640682,7300,950,50,48.00,50,79.95,22237.4,CE


In [103]:
df[df['instrumentTyoe']=='PE'].head(2)

,strikePrice,expiryDate,underlying,openInterest,changeinOpenInterest,pchangeinOpenInterest,totalTradedVolume,impliedVolatility,lastPrice,change,pChange,totalBuyQuantity,totalSellQuantity,bidQty,bidprice,askQty,askPrice,underlyingValue,instrumentTyoe
0,22400,2024-04-04,NIFTY,0.0,0,0.0,0,0.0,0.0,0.0,0.0,900,950,900,375.15,50,405.3,22237.4,PE
4,20650,2024-04-04,NIFTY,0.0,0,0.0,0,0.0,0.0,0.0,0.0,4500,0,750,5.10,0,0.0,22237.4,PE


In [104]:
ce_data=df[(df['expiryDate']==EX) & (df['instrumentTyoe']=="CE")]
ce_data=ce_data.sort_values(by='strikePrice')

In [105]:
ce_data.reset_index(drop=True,inplace=True)

In [106]:
ce_data.tail(5)

,strikePrice,expiryDate,underlying,openInterest,changeinOpenInterest,pchangeinOpenInterest,totalTradedVolume,impliedVolatility,lastPrice,change,pChange,totalBuyQuantity,totalSellQuantity,bidQty,bidprice,askQty,askPrice,underlyingValue,instrumentTyoe
102,23750,2024-02-29,NIFTY,6394.0,5084,388.091603,22370,18.99,0.70,-0.55,-44.000000,988950,231900,125750,0.70,63900,0.75,22237.4,CE
103,23800,2024-02-29,NIFTY,21069.0,6354,43.180428,47090,19.68,0.75,-0.45,-37.500000,2072000,351800,210250,0.70,58850,0.75,22237.4,CE
104,23850,2024-02-29,NIFTY,13956.0,6715,92.735810,29583,20.09,0.65,-0.45,-40.909091,1515550,305750,318050,0.65,58350,0.70,22237.4,CE
105,23900,2024-02-29,NIFTY,20272.0,13536,200.950119,65835,20.48,0.70,-0.50,-41.666667,2286150,669600,77900,0.65,336800,0.70,22237.4,CE
106,23950,2024-02-29,NIFTY,31966.0,13883,76.773765,75978,21.18,0.65,-0.55,-45.833333,2285200,635750,1028300,0.60,111300,0.65,22237.4,CE


In [107]:
ce_data.columns

Index(['strikePrice', 'expiryDate', 'underlying', 'openInterest',
       'changeinOpenInterest', 'pchangeinOpenInterest', 'totalTradedVolume',
       'impliedVolatility', 'lastPrice', 'change', 'pChange',
       'totalBuyQuantity', 'totalSellQuantity', 'bidQty', 'bidprice', 'askQty',
       'askPrice', 'underlyingValue', 'instrumentTyoe'],
      dtype='object')

In [108]:
pe_data=df[(df['expiryDate']==EX) & (df['instrumentTyoe']=="PE")]
pe_data=pe_data.sort_values(by='strikePrice')

In [109]:
pe_data.reset_index(drop=True,inplace=True)

In [110]:
pe_data

,strikePrice,expiryDate,underlying,openInterest,changeinOpenInterest,pchangeinOpenInterest,totalTradedVolume,impliedVolatility,lastPrice,change,pChange,totalBuyQuantity,totalSellQuantity,bidQty,bidprice,askQty,askPrice,underlyingValue,instrumentTyoe
0,18650,2024-02-29,NIFTY,17741.0,15732,783.076157,68910,47.18,0.60,-0.40,-40.000000,974000,327150,15750,0.60,106450,0.65,22237.4,PE
1,18700,2024-02-29,NIFTY,2861.0,1385,93.834688,7864,46.86,0.65,-0.30,-31.578947,120050,51650,21000,0.60,7550,0.65,22237.4,PE
2,18750,2024-02-29,NIFTY,1002.0,937,1441.538462,3534,46.52,0.65,-0.50,-43.478261,88150,19700,12500,0.65,300,0.70,22237.4,PE
3,18800,2024-02-29,NIFTY,1207.0,807,201.750000,4037,45.86,0.65,-0.40,-38.095238,99300,42350,7050,0.65,5000,0.70,22237.4,PE
4,18850,2024-02-29,NIFTY,673.0,568,540.952381,1629,44.89,0.70,-0.40,-36.363636,21650,28400,1250,0.65,750,0.70,22237.4,PE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,23750,2024-02-29,NIFTY,60.0,0,0.000000,0,0.00,0.00,0.00,0.000000,13500,13500,1800,1476.40,1800,1523.30,22237.4,PE
103,23800,2024-02-29,NIFTY,55.0,0,0.000000,0,0.00,0.00,0.00,0.000000,12500,12500,600,1493.75,600,1583.90,22237.4,PE
104,23850,2024-02-29,NIFTY,74.0,0,0.000000,1,0.00,1517.95,-192.95,-11.277690,13050,13100,250,1544.95,50,1609.25,22237.4,PE
105,23900,2024-02-29,NIFTY,6.0,0,0.000000,0,0.00,0.00,0.00,0.000000,10500,10500,600,1593.50,600,1683.90,22237.4,PE


In [111]:
ce_data

,strikePrice,expiryDate,underlying,openInterest,changeinOpenInterest,pchangeinOpenInterest,totalTradedVolume,impliedVolatility,lastPrice,change,pChange,totalBuyQuantity,totalSellQuantity,bidQty,bidprice,askQty,askPrice,underlyingValue,instrumentTyoe
0,18650,2024-02-29,NIFTY,657.0,0,0.000000,8,69.21,3626.30,118.70,3.384080,10100,9850,50,3571.60,50,3621.20,22237.4,CE
1,18700,2024-02-29,NIFTY,1353.0,-39,-2.801724,44,0.00,3543.35,13.35,0.378187,10350,9650,50,3543.00,500,3557.55,22237.4,CE
2,18750,2024-02-29,NIFTY,23.0,0,0.000000,0,0.00,0.00,0.00,0.000000,9900,9950,1800,3453.40,50,3540.85,22237.4,CE
3,18800,2024-02-29,NIFTY,223.0,0,0.000000,0,0.00,0.00,0.00,0.000000,10100,8400,1800,3403.75,50,3492.20,22237.4,CE
4,18850,2024-02-29,NIFTY,0.0,0,0.000000,0,0.00,0.00,0.00,0.000000,9900,8250,1800,3350.35,50,3441.10,22237.4,CE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,23750,2024-02-29,NIFTY,6394.0,5084,388.091603,22370,18.99,0.70,-0.55,-44.000000,988950,231900,125750,0.70,63900,0.75,22237.4,CE
103,23800,2024-02-29,NIFTY,21069.0,6354,43.180428,47090,19.68,0.75,-0.45,-37.500000,2072000,351800,210250,0.70,58850,0.75,22237.4,CE
104,23850,2024-02-29,NIFTY,13956.0,6715,92.735810,29583,20.09,0.65,-0.45,-40.909091,1515550,305750,318050,0.65,58350,0.70,22237.4,CE
105,23900,2024-02-29,NIFTY,20272.0,13536,200.950119,65835,20.48,0.70,-0.50,-41.666667,2286150,669600,77900,0.65,336800,0.70,22237.4,CE


In [112]:

def tab(t):
    t=dict(t)
    instruments=list(t.keys())
    value=list(t.values())
    head=['INSTRUMENTS','VALUE']
    data={'instruments':instruments,'value':value}
    table = tabulate(data, headers=head, tablefmt="grid")
    return(print(table))

In [113]:
tab(ce_data.iloc[-1])

+-----------------------+---------------------+
| INSTRUMENTS           | VALUE               |
+=======================+=====================+
| strikePrice           | 23950               |
+-----------------------+---------------------+
| expiryDate            | 2024-02-29 00:00:00 |
+-----------------------+---------------------+
| underlying            | NIFTY               |
+-----------------------+---------------------+
| openInterest          | 31966.0             |
+-----------------------+---------------------+
| changeinOpenInterest  | 13883               |
+-----------------------+---------------------+
| pchangeinOpenInterest | 76.7737654150307    |
+-----------------------+---------------------+
| totalTradedVolume     | 75978               |
+-----------------------+---------------------+
| impliedVolatility     | 21.18               |
+-----------------------+---------------------+
| lastPrice             | 0.65                |
+-----------------------+---------------

In [114]:
ce_data["impliedVolatility"].iloc[-1]

21.18

In [115]:
def black_scholes(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call_price

def calculate_option_price(underlying_price, strike_price, time_to_expiration, risk_free_rate, implied_volatility):
    
    option_price = black_scholes(underlying_price, strike_price, time_to_expiration, risk_free_rate, implied_volatility)
    return option_price


underlying_price =ce_data['underlyingValue'].iloc[-1]
strike_price = ce_data['strikePrice'].iloc[-1]
today = datetime.now()
expiry_date=ce_data['expiryDate'].iloc[-1]
time_to_expiration = (expiry_date - today)/timedelta(days=1)/365
risk_free_rate = 0.1
implied_volatility =(ce_data['impliedVolatility'].iloc[-1])/100

option_price = calculate_option_price(underlying_price, strike_price, time_to_expiration, risk_free_rate, implied_volatility)


print("Option Price:", option_price)


Option Price: 0.4091660998292568


In [116]:
from py_vollib.black_scholes.greeks.analytical import delta,gamma,rho,theta

In [117]:
underlying_price =ce_data['underlyingValue'].iloc[-1]
strike_price = ce_data['strikePrice'].iloc[-1]
today = datetime.now()
expiry_date=ce_data['expiryDate'].iloc[-1]
time_to_expiration = (expiry_date - today)/timedelta(days=1)/365
risk_free_rate = 0.1
implied_volatility =(ce_data['impliedVolatility'].iloc[-1])/100

In [118]:
d=delta('c',underlying_price,strike_price,time_to_expiration,risk_free_rate,implied_volatility)
d

0.002430369790886762

In [119]:
g=gamma('c',underlying_price,strike_price,time_to_expiration,risk_free_rate,implied_volatility)
g

1.3234522730545614e-05

In [120]:
r=rho('c',underlying_price,strike_price,time_to_expiration,risk_free_rate,implied_volatility)
r

0.007900444030410058

In [121]:
t=theta('c',underlying_price,strike_price,time_to_expiration,risk_free_rate,implied_volatility)
t

-0.4168607780790276